In [1]:
import yfinance as yf
import datetime as dt
import requests, json, pandas as pd
from bs4 import BeautifulSoup
from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import pandas as pd
import os
import shutil

PYSPARK_PYTHON = os.getenv("PYSPARK_PYTHON") 
PYSPARK_DRIVER_PYTHON = os.getenv("PYSPARK_DRIVER_PYTHON")
import pyspark
from delta import configure_spark_with_delta_pip, DeltaTable
import json

# Load the configuration JSON file
with open('/usr/local/spark/conf/spark-defaults.json', 'r') as f:
    config = json.load(f)

# Initialize the Spark session builder
builder = pyspark.sql.SparkSession.builder.appName("MyApp1").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").config("spark.pyspark.python", PYSPARK_PYTHON)\
    .config("spark.pyspark.driver.python", PYSPARK_DRIVER_PYTHON)

# Read the packages from the text file
packages = []
with open('/usr/local/spark/conf/packages.txt', 'r') as file:
    # Read each line and strip newlines or extra spaces
    packages = [line.strip() for line in file if line.strip()]

# # Add packages to the Spark session configuration
builder.config("spark.jars.packages", ",".join(packages))

# Apply the configurations from the JSON file to the Spark session
for key, value in config.items():
    builder.config(key, value)

# Configure Spark with Delta Lake (if needed)
spark = configure_spark_with_delta_pip(builder).getOrCreate()
# Now you can use the Spark session
spark

In [2]:
dfpd=pd.read_csv('/app/Notebooks/Config_Stock.csv')
dfpath=spark.createDataFrame(dfpd)

In [3]:

trgt_path_processed = dfpath.filter(F.col("DataFeedName") == "Stock_Delta_Path").select('Path').collect()[0][0]
trgt_path_csv = dfpath.filter(F.col("DataFeedName") == "Stock_CSV_Path").select('Path').collect()[0][0]

In [4]:
def f_roe(financials_info, balance_sheet):
    net_income = financials_info.loc["Net Income"].iloc[0] if "Net Income" in financials_info.index else None
    # Fetch Shareholders' Equity from balance sheet
    total_equity = balance_sheet.loc["Stockholders Equity"].iloc[0] if "Stockholders Equity" in balance_sheet.index else None 
    return (net_income / total_equity) * 100 if type(net_income) == float and type(total_equity) == float else 0.00

In [5]:
def f_roce(financials_info, balance_sheet):
    ebit = financials_info.loc["Operating Income"].iloc[0] if "Operating Income" in financials_info.index else 0

    # # Get Total Assets and Current Liabilities from balance sheet
    total_assets = balance_sheet.loc["Total Assets"].iloc[0] if "Total Assets" in balance_sheet.index else 0
    current_liabilities = balance_sheet.loc["Current Liabilities"].iloc[0] if "Current Liabilities" in balance_sheet.index else 0

    # Calculate Capital Employed
    capital_employed = total_assets - current_liabilities

    return (ebit / capital_employed) * 100 if capital_employed != 0 else 0

In [6]:
def f_PEG(stock_info):
# Calculate PEG ratios
    trailing_pe = stock_info.get("trailingPE", None)
    forward_pe = stock_info.get("forwardPE", None)
    earnings_growth = stock_info.get("earningsGrowth", None)  # Provided as a decimal

    if earnings_growth is not None and earnings_growth > 0:
        
        trailing_peg = trailing_pe / (earnings_growth * 100) if trailing_pe else 0
        forward_peg = forward_pe / (earnings_growth * 100) if forward_pe else 0
        peg_t= trailing_peg if trailing_peg else "N/A"
        peg_f=forward_peg if forward_peg else "N/A"
    else:
        peg_f=peg_t="N/A"
    return peg_t,peg_f

In [7]:
def f_debt_to_equity(balance_sheet):
    total_liabilities = balance_sheet.loc["Total Liabilities Net Minority Interest"].iloc[0] if "Total Liabilities Net Minority Interest" in balance_sheet.index else 0
    shareholders_equity = balance_sheet.loc["Stockholders Equity"].iloc[0] if "Stockholders Equity" in balance_sheet.index else 0
    # Calculate Debt-to-Equity Ratio
    if shareholders_equity != 0:  # Avoid division by zero
        debt_to_equity_ratio = total_liabilities / shareholders_equity
    else:
        debt_to_equity_ratio = "N/A"
    return debt_to_equity_ratio

In [8]:
def f_sales_growth(income_statement):
    revenue = income_statement.loc["Total Revenue"] if "Total Revenue" in income_statement.index else {"0":"NA"}
    revenue = revenue.dropna() if isinstance(revenue, pd.Series) else revenue # Remove any periods with missing data
    # Ensure revenue has at least two periods to calculate growth
    if len(revenue) > 1:
        # Calculate sales growth between the latest two periods
        latest_growth = ((revenue.iloc[0] - revenue.iloc[1]) / revenue.iloc[1]) * 100 if revenue.iloc[1] != 0 else 0
        latest_period = revenue.index[0].strftime("%Y-%m-%d")
    else:
        # Handle cases where there isn't enough data
        latest_growth=0
        latest_period=0
    return latest_growth,latest_period

In [9]:
def f_MA(historical_data):
    # Calculate 50-day and 200-day moving averages
    if not historical_data.empty:
        historical_data["MA50"] = historical_data["Close"].rolling(window=50).mean()
        historical_data["MA200"] = historical_data["Close"].rolling(window=200).mean()
        # Return the latest MA50 and MA200
        latest_data = historical_data.iloc[-1]
        ma50=latest_data["MA50"] if latest_data["MA50"] else 0
        ma200=latest_data["MA200"] if latest_data["MA200"] else 0
    else:
        # Return the latest MA50 and MA200
        latest_data = 0
        ma50=0
        ma200=0
    return ma50, ma200

In [10]:
def f_stock_data(l_tickers):
    ticker_data = []
    headers=["Ticker","Sector","Industry","fiftytwo_week_high","ROE","ROCE","Trailing_PEG","Forward_PEG","Debt_to_Equity","Latest_Finanacial_Year","Sales_Growth","MA50","MA200"]
    for t in l_tickers:
        ticker = yf.Ticker(t + ('.BO' if t.isdigit() else '.NS'))
        stock_info = ticker.info
        balance_sheet = ticker.balance_sheet
        financials_info=ticker.financials
        income_statement=ticker.income_stmt
        historical_data = ticker.history(period="ytd")
        v_roe=f_roe(financials_info, balance_sheet)
        v_roce=f_roce(financials_info, balance_sheet)
        v_peg_t,v_peg_f=f_PEG(stock_info)
        v_debt_to_equity=f_debt_to_equity(balance_sheet)
        v_sales_growth,v_latest_period=f_sales_growth(income_statement)
        v_ma50,v_ma200=f_MA(historical_data)
        ticker_data.append([t, stock_info.get("sector", "N/A"), stock_info.get("industry", "N/A"), stock_info.get("fiftyTwoWeekHigh", None),v_roe,v_roce,v_peg_t,v_peg_f,v_debt_to_equity,v_latest_period,v_sales_growth,v_ma50,v_ma200])
    df_retun = pd.DataFrame(ticker_data, columns=headers)
    return(df_retun)

In [11]:
df_pd=pd.read_csv("/home/jovyan/Notebooks/Holiday.csv")
df_holday_csv=spark.createDataFrame(df_pd)
holiday="('" + "','".join([row["Date"] for row in df_holday_csv.select('Date').collect()]) + "')"

In [12]:
#To prevent Data overwriting on Market Hours, Preventing the Auto run of this notebook on NSE and BSE timings (9 AM to 4 PM IST)
# Get the current date and time
# current_timestamp = dt.datetime.now()
# current_day=current_timestamp.strftime("%d-%b-%Y")
# week_day=current_timestamp.strftime('%A')
# # Replace the time portion with 03:30:00 and 10:30:00
# final_timestamp = current_timestamp.replace(hour=3, minute=30, second=0, microsecond=0)
# end_timestamp = current_timestamp.replace(hour=10, minute=30, second=0, microsecond=0)

# print(current_timestamp, current_day, week_day)

# if current_day in holiday:
#     print(f"Today {current_day} is a Holiday of {df_holday_csv.filter(F.col('Date') == current_day).select('Description').collect()[0][0]}\nExiting...")
#     exit()
# elif week_day in ['Saturday','Sunday']:
#     print(f"Today is {week_day}\nExiting...")
#     exit()
# elif final_timestamp <= current_timestamp <= end_timestamp:
#     print(f"The current time {current_timestamp.time()} is between the start {final_timestamp.time()} and end {end_timestamp.time()} timestamps. Market is LIVE NOW \nExiting...")
#     exit()
# elif current_timestamp < final_timestamp:
#     v_yesterday=v_yesterday=(current_timestamp - dt.timedelta(days=1)).date().strftime("%d-%b-%Y")
#     print(v_yesterday)
#     if v_yesterday in holiday:
#         print(f"Yesterday {current_timestamp.date()} was Holiday of {df_holday_csv.filter(F.col('Date') == current_day).select('Description').collect()[0][0]}")
#         exit()
#     elif v_yesterday in ['Saturday','Sunday']:
#         print(f"Yesterday is {week_day}\nExiting...")
#         exit()
#     else:
#         print(f"The current time {current_timestamp.time()} is NOT between the start {final_timestamp.time()} and end {end_timestamp.time()} timestamps. Market is CLOSED NOW\nContinuing Execution...")
# else:
#     print(f"The current time {current_timestamp.time()} is NOT between the start {final_timestamp.time()} and end {end_timestamp.time()} timestamps. Market is CLOSED NOW\nContinuing Execution...")


In [13]:
# v_yesterday=(current_timestamp - dt.timedelta(days=1)).date().strftime("%d-%b-%Y")
# v_day=v_yesterday
# print(v_yesterday)

In [14]:
# URL to scrape
url_link=["https://www.google.com/finance/markets/gainers","https://www.google.com/finance/markets/losers"]
rows = []
headers=["Ticker","Stock_Name","CMP","Change","Change_Percentage"]

for url in url_link:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find the parent container
        parent_container = soup.find('ul', class_='sbnBtf')
        if parent_container:
            # Find all stock entries within the parent container
            stock_entries = parent_container.find_all('li')
            for stock in stock_entries:
                # Extract relevant details for each stock
                stock_ticker = stock.find('div', class_='COaKTb').text if stock.find('div', class_='COaKTb') else "N/A"
                stock_name = stock.find('div', class_='ZvmM7').text if stock.find('div', class_='ZvmM7') else "N/A"
                stock_price = stock.find('div', class_='YMlKec').text if stock.find('div', class_='YMlKec') else "N/A"
                stock_change = stock.find('div', class_='BAftM').text if stock.find('div', class_='BAftM') else "N/A"
                stock_percent = stock.find('div', class_='zWwE1').text if stock.find('div', class_='zWwE1') else "N/A"
                # Add extracted data to the list
                rows.append([stock_ticker,stock_name,stock_price,stock_change,stock_percent])
    
    # Convert to JSON string with readable characters
df_pd_today = pd.DataFrame(rows, columns=headers)


In [15]:
# Extract unique tickers as a Python list
l_tickers = df_pd_today["Ticker"].unique().tolist()
# Print the result
print(l_tickers)
df_stock_data=f_stock_data(l_tickers)

In [16]:
df_custom=df_pd_today.merge(df_stock_data, on='Ticker',how='left')
reorder_colms=["Ticker","Stock_Name","Sector","Industry","CMP","Change","Change_Percentage"]+[col for col in df_custom.columns if col not in ["Ticker","Stock_Name","Sector","Industry","CMP","Change","Change_Percentage"]]
df_spark=spark.createDataFrame(df_custom[reorder_colms])

In [57]:
df_master = df_spark.filter(
                            (F.col("ROE") >= 15) &
                            (F.col("ROCE") >= 15) &
                            (F.col("Debt_to_Equity") <= 1) &
                            (F.col("MA50") >= F.col("MA200"))
                            ).withColumn("ROE",F.coalesce(F.round(F.col("ROE"), 2),F.lit(0))) \
                            .withColumn("ROCE", F.coalesce(F.round(F.col("ROCE"), 2),F.lit(0))) \
                            .withColumn("Trailing_PEG", F.coalesce(F.round(F.col("Trailing_PEG"), 2),F.lit(0))) \
                            .withColumn("Forward_PEG", F.coalesce(F.round(F.col("Forward_PEG"), 2),F.lit(0))) \
                            .withColumn("Debt_to_Equity", F.coalesce(F.round(F.col("Debt_to_Equity"), 2),F.lit(0))) \
                            .withColumn("Sales_Growth", F.coalesce(F.round(F.col("Sales_Growth"), 2), F.lit(0))) \
                            .withColumn("MA50", F.coalesce(F.round(F.col("MA50"), 2), F.lit(0))) \
                            .withColumn("MA200", F.coalesce(F.round(F.col("MA200"), 2), F.lit(0))) \
                            .withColumn(
                                "Gainer_Looser",
                                F.when(
                                    F.regexp_replace(F.col("Change"), "₹", "").cast("float") < 0.0, "L"
                                ).otherwise("G")
                            )

In [58]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    df_read = spark.read.format('delta').load(trgt_path_processed)
    df_repeat=df_read.join(df_master.select('Ticker'), on='Ticker', how='inner')\
            .withColumn('RunTimeStamp',(F.current_timestamp()).cast("timestamp"))\
            .withColumn("NextDay", F.to_timestamp(F.date_format(F.date_add(F.col("UpdateTimestamp"), 1), format="yyyy-MM-dd 10:30:00")))\
            .withColumn("WatchOutFlag", F.when(
                F.col('RunTimeStamp') > F.col('NextDay'), (F.col('WatchOutFlag') + 1)).otherwise(F.col('WatchOutFlag'))) \
            .withColumn("UpdateTimestamp",  F.when(
                        F.col('RunTimeStamp') > F.col('NextDay'),
                        F.to_timestamp(F.date_format(F.current_date(), format="yyyy-MM-dd 10:30:00"))).otherwise(F.col('UpdateTimestamp')))\
            .drop("RunTimeStamp","NextDay","PKSK","RowSK")
    replace_tickers = df_repeat.select("Ticker").rdd.flatMap(lambda x: x).collect()
    df_output = (df_master.filter(~F.col("Ticker").isin(replace_tickers))).unionByName(df_repeat, allowMissingColumns=True) \
        .withColumn("WatchOutFlag", F.coalesce(F.col("WatchOutFlag"), F.lit(0)).cast('int'))\
        .withColumn('UpdateTimestamp', F.coalesce(F.col("UpdateTimestamp"),F.date_format(F.current_timestamp(), format="yyyy-MM-dd 10:30:00")))   
else:
    df_output=df_master.withColumn('WatchOutFlag',F.lit(0))\
                .withColumn('UpdateTimestamp', F.date_format(F.current_timestamp(), format="yyyy-MM-dd 10:30:00"))
df_final=df_output.withColumn("PKSK", F.xxhash64(F.col("Ticker")).cast("string"))\
        .withColumn("RowSK", F.xxhash64(F.concat_ws("|", *[F.col(c) for c in df_output.columns])))

In [59]:
df_final.createOrReplaceTempView('vw_source')
dup_query="select PKSK from vw_source group by 1 having count(PKSK)>1 "
df_dup=spark.sql(dup_query)
x = df_dup.count()
if x == 0:
    print("No duplicates.. continuing execution")
else:
    print(f"Exception of duplicates! Found {x} duplicate keys.")
    # #exit()

In [60]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    # First delete records where both PKSK and RowSK match
    deletion_query = f"""
    MERGE INTO delta.`{trgt_path_processed}` AS target
    USING vw_source AS source
    ON target.PKSK = source.PKSK AND target.RowSK = source.RowSK
    WHEN MATCHED THEN DELETE
    """
    spark.sql(deletion_query)
    
    # Then handle updates (different RowSK) and inserts
    column_name = df_final.columns
    set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
    insert_clause = ",".join(column_name)
    insert_values = ",".join([f"source.{i}" for i in column_name])
    
    merge_query = f"""
    MERGE INTO delta.`{trgt_path_processed}` AS target
    USING vw_source AS source
    ON target.PKSK = source.PKSK
    WHEN MATCHED AND target.RowSK <> source.RowSK THEN
        UPDATE SET {set_clause}
    WHEN NOT MATCHED THEN
        INSERT ({insert_clause}) VALUES ({insert_values})
    """
    spark.sql(merge_query)     
else :
    query=f"CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source"
    spark.sql(query)

print(query)

In [61]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)
delta_table=DeltaTable.forPath(spark, trgt_path_processed)
delta_table.vacuum(retentionHours=0)
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", True)


In [62]:
df_read = spark.read.format('delta').load(trgt_path_processed)
display(df_read)

In [63]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1)\
    .write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)

In [64]:
trgt_copy_path = trgt_path_csv + "processed.csv"
files=os.listdir(trgt_path_csv)
selected_files = [file for file in files if file.startswith('part-00') and file.endswith('.csv')]
file=trgt_path_csv + selected_files[0]
print(selected_files)
shutil.copy(file, trgt_copy_path)

In [65]:
delete_log = [file for file in files if "processed.csv" != file ]
for file in delete_log :
    os.remove(trgt_path_csv + file)
    print(f"removed {trgt_path_csv + file}")